# 第7回講義 宿題

## 課題. Tensorflowを用いて, MNISTを畳み込みニューラルネットワーク(CNN)で学習せよ

### 注意

- homework関数を完成させて提出してください
    - 訓練データはtrain_X, train_y, テストデータはtest_Xで与えられます
    - train_Xとtrain_yをtrain_X, train_yとvalid_X, valid_yに分けるなどしてモデルを学習させてください
    - test_Xに対して予想ラベルpred_yを作り, homework関数の戻り値としてください\
- pred_yのtest_yに対する精度(F値)で評価します
- 全体の実行時間がiLect上で60分を超えないようにしてください
- homework関数の外には何も書かないでください (必要なものは全てhomework関数に入れてください)
- 解答提出時には Answer Cell の内容のみを提出してください

**`tf` の以下のモジュールはこの回では使用できないように制限されています. 注意してください.**
```python
tf.app
tf.compat
tf.contrib
tf.erros
tf.gfile
tf.graph_util
tf.image
tf.layers
tf.logging
tf.losses
tf.metrics
tf.python_io
tf.resource_loader
tf.saved_model
tf.sdca
tf.sets
tf.summary
tf.sysconfig
tf.test
```

次のセルのhomework関数を完成させて提出してください

# Answer Cell

- 以下のvalidate_homework関数を用いてエラーが起きないか動作確認をして下さい。
- 提出に際して、以下のscore_homework関数で60分で実行が終わることを確認して下さい。
- 評価は以下のscore_homework関数で行われますが、random_stateの値は変更されます。

# Checker Cell (for student)

In [1]:
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tensorflow.examples.tutorials.mnist import input_data

import numpy as np
import tensorflow as tf

del [
    tf.app,
    tf.compat,
    tf.contrib,
    tf.errors,
    tf.gfile,
    tf.graph_util,
    tf.image,
    tf.layers,
    tf.logging,
    tf.losses,
    tf.metrics,
    tf.python_io,
    tf.resource_loader,
    tf.saved_model,
    tf.sdca,
    tf.sets,
    tf.summary,
    tf.sysconfig,
    tf.test
]

def load_mnist():
    mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
    mnist_X = np.r_[mnist.train.images, mnist.test.images]
    mnist_y = np.r_[mnist.train.labels, mnist.test.labels]
    return train_test_split(mnist_X, mnist_y, test_size=0.2, random_state=42)

def validate_homework():
    train_X, test_X, train_y, test_y = load_mnist()
    train_X = train_X.reshape((train_X.shape[0], 28, 28, 1))
    test_X = test_X.reshape((test_X.shape[0], 28, 28, 1))

    # validate for small dataset
    train_X_mini = train_X[:1000]
    train_y_mini = train_y[:1000]
    test_X_mini = test_X[:100]
    test_y_mini = test_y[:100]

    pred_y = homework(train_X_mini, train_y_mini, test_X_mini)
    print(pred_y)
    print(f1_score(np.argmax(test_y_mini, 1), pred_y, average='macro'))

def score_homework():
    train_X, test_X, train_y, test_y = load_mnist()
    train_X = train_X.reshape((train_X.shape[0], 28, 28, 1))
    test_X = test_X.reshape((test_X.shape[0], 28, 28, 1))
    
    pred_y = homework(train_X, train_y, test_X)
    print(f1_score(np.argmax(test_y, 1), pred_y, average='macro'))

In [1]:
def homework(train_X, train_y, test_X):
    
    import numpy as np
    import tensorflow as tf 
    from sklearn.utils import shuffle
    from sklearn.metrics import f1_score
    from sklearn.datasets import fetch_mldata
    from sklearn.model_selection import train_test_split
    
    # hyperparameter
    rng = np.random.RandomState(1234)
    random_state = 42
    
    # method
    def one_to_hot(x):
        """one to hot method"""
        columns = np.unique(x)
        X = np.zeros([x.shape[0], len(columns)])
        for i, column in enumerate(columns):
            X[np.where(x==column), i] = 1
        return X

    class EarlyStopping(object):
        """early stopping"""
        
        def __init__(self, limit=15):
            self.stop_count = 0
            self.limit = limit
            self.best_validation_loss = float('inf')
            
        def check(self, loss):
            if loss < self.best_validation_loss:
                self.best_validation_loss = loss
                self.stop_count = 0
            else:
                self.stop_count += 1
            
            if self.stop_count > self.limit:
                return True
            else:
                return False

    class Conv:
        def __init__(self, filter_shape, function=lambda x: x, strides=[1,1,1,1], padding="SAME"):
            # Xavier Initialization
            fan_in = np.prod(filter_shape[:3])
            fan_out = np.prod(filter_shape[:2]) * filter_shape[3]
            self.W = tf.Variable(rng.uniform(
                            low=-np.sqrt(6/(fan_in + fan_out)),
                            high=np.sqrt(6/(fan_in + fan_out)),
                            size=filter_shape
                        ).astype('float32'), name='W')
            self.b = tf.Variable(np.zeros((filter_shape[3]), dtype='float32'), name='b') # バイアスはフィルタごとなので, 出力フィルタ数と同じ次元数
            self.function = function
            self.strides = strides
            self.padding = padding

        def f_prop(self, x):
            u = tf.nn.conv2d(x, self.W, strides=self.strides, padding=self.padding) + self.b
            return self.function(u)

    class Pooling:
        def __init__(self, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME"):
            self.ksize = ksize
            self.strides = strides
            self.padding = padding

        def f_prop(self, x):
            return tf.nn.max_pool(x, ksize=self.ksize, strides=self.strides, padding=self.padding)
        
    class Flatten:
        def f_prop(self, x):
            # prod -> 指定方向の配列の要素の積
            return tf.reshape(x, (-1, np.prod(x.get_shape().as_list()[1:])))
        
    class Dense:
        def __init__(self, in_dim, out_dim, function=lambda x: x):
            # Xavier Initialization
            self.W = tf.Variable(rng.uniform(
                            low=-np.sqrt(6/(in_dim + out_dim)),
                            high=np.sqrt(6/(in_dim + out_dim)),
                            size=(in_dim, out_dim)
                        ).astype('float32'), name='W')
            self.b = tf.Variable(np.zeros([out_dim]).astype('float32'))
            self.function = function

        def f_prop(self, x):
            return self.function(tf.matmul(x, self.W) + self.b)
        
    
    class ConvolutionNeuralNetwork:
    
        def __init__(self, layers):
            """
            # :param layers: = [                            # (縦の次元数)x(横の次元数)x(チャネル数)
            #                 Conv((5, 5, 1, 20), tf.nn.relu),  # 28x28x 1 -> 24x24x20
            #                Pooling((1, 2, 2, 1)),            # 24x24x20 -> 12x12x20
            #                 Conv((5, 5, 20, 50), tf.nn.relu), # 12x12x20 ->  8x 8x50
            #                 Pooling((1, 2, 2, 1)),            #  8x 8x50 ->  4x 4x50
            #                Flatten(),
            #                 Dense(4*4*50, 10, tf.nn.softmax)
            # ]
            
            """
            self.layers = layers
            # start session
            self.sess = tf.Session()
            self.sess.run(tf.global_variables_initializer())
            
            self.n_epochs = 150
            self.batch_size = 100
            self.n_batches = train_X.shape[0] // self.batch_size
            
        def __call__(self, input_shape=[None, 28, 28, 1], target_shape=[None, 10]):
            # define graph
            self.x = tf.placeholder(tf.float32, input_shape)
            self.t = tf.placeholder(tf.float32, target_shape)
            self.keep_prob = tf.placeholder(tf.float32)
            
            #self.cost = -tf.reduce_mean(tf.reduce_sum(self.t * tf.log(self.y), 1))
            self.y = self.f_props(self.layers, self.x)
            loss = tf.nn.softmax_cross_entropy_with_logits(logits=self.y, labels=self.t)
            self.cross_entropy = tf.reduce_mean(loss)
            # self.cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
            self.train_step = tf.train.AdamOptimizer(1e-4).minimize(self.cross_entropy)
            
            
            self.valid = tf.argmax(self.y, 1)
            self.sess.run(tf.global_variables_initializer())
            
        def f_props(self, layers, x):
            params = []
            for idx, layer in enumerate(layers):
                # dropout
                if idx > 0:
                    x = tf.nn.dropout(x, self.keep_prob)
                x = layer.f_prop(x)
                # これextendかな?
            return x
        
        def _train(self, train_X, train_y):
            
            # start learning
            for epoch in range(self.n_epochs):
                train_X, train_y = shuffle(train_X, train_y, random_state=random_state)
                # batch_train_X, batch_valid_X, batch_train_y, batch_valid_y = train_test_split(train_X, train_y, train_size=0.8, random_state=random_state)
                
                for i in range(self.n_batches):
                    # print(i)
                    start = i * self.batch_size
                    end = start + self.batch_size
                    self.sess.run(self.train_step, feed_dict={self.x: train_X[start:end], self.t: train_y[start:end], self.keep_prob: 0.8})
                                 
        def predict(self, test_X):
            return self.sess.run(self.valid, feed_dict={self.x: test_X, self.keep_prob: 1.0})

    # main
    #  train_y = one_to_hot(train_y)
    layers = [                            # (縦の次元数)x(横の次元数)x(チャネル数)
        Conv((5, 5, 1, 32), tf.nn.relu),  # 28x28x 1 -> 24x24x32
        Pooling((1, 2, 2, 1)),            # 24x24x32 -> 12x12x32
        Conv((5, 5, 32, 64), tf.nn.relu), # 12x12x32 ->  8x 8x64
        Pooling((1, 1, 1, 1)),            #  8x 8x64 ->  4x 4x64
        Flatten(),
        Dense(7*7*64, 1024, tf.nn.relu),
        Dense(1024, 10, tf.nn.softmax)
    ]
    
    model = ConvolutionNeuralNetwork(layers)
    model()
    model._train(train_X, train_y)
    pred_y = model.predict(test_X)
    model.sess.close()
    
    return pred_y

In [3]:
validate_homework()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
[6 6 9 7 0 0 3 1 3 1 3 0 2 3 2 4 9 1 2 6 8 6 9 2 6 6 7 6 7 6 2 1 3 5 4 3 9
 0 0 0 0 8 1 6 7 6 7 9 9 5 5 2 5 1 0 1 4 1 2 0 2 1 1 9 2 3 1 8 6 0 5 7 7 0
 6 5 7 3 5 1 0 3 8 6 0 3 7 3 7 8 6 7 7 7 0 2 0 0 1 5]
0.946031746032
